In [335]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [336]:
# Starting from last days of 2022 so that indicators work properly on data from starting of 2023.
data = yf.download('TCS.NS', start='2022-12-10', end='2023-12-31')
df = pd.DataFrame(data)

/tmp/ipython-input-2283485118.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download('TCS.NS', start='2022-12-10', end='2023-12-31')
[*********************100%***********************]  1 of 1 completed


In [337]:
def rsi(df, period=14):
  gains = df.diff(1)
  gains = gains.where(gains > 0, 0)
  losses = df.diff(1)
  losses = losses.where(losses < 0, 0)
  avg_gains = gains.rolling(window=period).mean()
  avg_losses = losses.rolling(window=period).mean().abs()

  df['rsi'] = 100-(100 / (1 + avg_gains/avg_losses))
  return df['rsi']

In [338]:
def bbands(df, period=20, std_deviation=2):
  sma = df.rolling(window=period).mean()
  std = df.rolling(window=period).std()
  upper_band = sma + (std_deviation * std)
  lower_band = sma - (std_deviation * std)
  return upper_band,sma,lower_band

In [339]:
def macd(df, signal_period=9, short_period=12, long_period=26):
  short_ema = df.ewm(span=short_period, adjust=False).mean()
  long_ema = df.ewm(span=long_period, adjust=False).mean()
  macd_line = short_ema - long_ema
  signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()
  histogram = macd_line - signal_line
  return macd_line, signal_line, histogram

In [340]:
def adx(df, period=14):
  df['+DI'] = 100 * (df['High'] - df['Low']) / (df['High'] - df['Low'])
  df['-DI'] = 100 * (df['Low'] - df['High']) / (df['High'] - df['Low'])
  df['+DI'] = df['+DI'].rolling(window=period).mean()
  df['-DI'] = df['-DI'].rolling(window=period).mean()
  df['TR'] = df['High'] - df['Low']
  df['ATR'] = df['TR'].rolling(window=period).mean()
  df['DX'] = (df['+DI'] - df['-DI']) / (df['+DI'] + df['-DI'])
  df['ADX'] = df['DX'].rolling(window=period).mean()
  return df['ADX']

In [341]:
def stochastic_oscillator(df, period=14, d_period=3):
  l14 = df['Low'].rolling(window=period).min()
  h14 = df['High'].rolling(window=period).max()
  df['%K'] = 100 * (df['Close'] - l14) / (h14 - l14)
  df['%D'] = df['%K'].rolling(window=d_period).mean()
  return df['%K'], df['%D']

In [342]:
def atr(df, period = 14):
  df['High-Low'] = df['High'] - df['Low']
  df['High-PrevClose'] = abs(df['High'] - df['Close'].shift(1))
  df['Low-PrevClose'] = abs(df['Low'] - df['Close'].shift(1))
  df['TR'] = df[['High-Low', 'High-PrevClose', 'Low-PrevClose']].max(axis=1)
  df['ATR'] = df['TR'].rolling(window=period).mean()

  return df['ATR']

**GENERATING SIGNALS:**

In [343]:
# RSI:
rsi_df = rsi(df['Close'])
df['RSI_Signal'] = np.where(rsi_df > 70, 1, np.where(rsi_df < 30, -1, 0))

/tmp/ipython-input-2757589409.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rsi'] = 100-(100 / (1 + avg_gains/avg_losses))


In [344]:
# Bollinger Bands:
upper_band, sma, lower_band = bbands(df['Close'])
df['BB_Signal'] = np.where(df['Close'] > upper_band, 1, np.where(df['Close'] < lower_band, -1, 0))

In [345]:
# MACD:
macd_line, signal_line, histogram = macd(df['Close'])
df['MACD_Signal'] = np.where(histogram > 0, 1, np.where(histogram < 0, -1, 0))

In [346]:
# SO:
k, d = stochastic_oscillator(df)
df['SO_Signal'] = np.where(k > d, 1, np.where(k < d, -1, 0))

In [347]:
# ADX:
adx(df)
df['ADX_Signal'] = np.where(df['ADX'] > 25, 1, np.where(df['ADX'] < 20, -1, 0))

In [348]:
# ATR:
df['ATR']=atr(df)
df['ATR_Signal'] = 0
df.loc[df['ATR'] > df['ATR'].shift(1), 'ATR_Signal'] = 1
df.loc[df['ATR'] < df['ATR'].shift(1), 'ATR_Signal'] = -1

In [349]:
final_df = df['2023-01-01':'2023-12-31']
final_df

Price,Close,High,Low,Open,Volume,RSI_Signal,BB_Signal,MACD_Signal,%K,%D,...,-DI,TR,ATR,DX,ADX,ADX_Signal,High-Low,High-PrevClose,Low-PrevClose,ATR_Signal
Ticker,TCS.NS,TCS.NS,TCS.NS,TCS.NS,TCS.NS,,,,,,...,,,,,,,,,,
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-02,2986.109131,2991.053382,2962.075230,2985.697166,709547,0,0,-1,45.840185,46.238507,...,-100.0,28.978152,47.521812,inf,NaN,0,28.978152,9.293372,19.684780,-1
2023-01-03,3031.796143,3039.715795,2971.322234,2977.456555,1245178,0,0,1,73.445889,54.299936,...,-100.0,68.393561,49.470674,inf,NaN,0,68.393561,53.606664,14.786897,1
2023-01-04,3034.817139,3046.445145,3008.769018,3027.538346,1231668,0,0,1,92.241769,70.509281,...,-100.0,37.676127,47.819361,inf,NaN,0,37.676127,14.649003,23.027124,-1
2023-01-05,3031.567627,3055.555704,3004.878419,3049.055017,1826057,0,0,1,84.912226,83.533295,...,-100.0,50.677285,46.213795,inf,NaN,0,50.677285,20.738565,29.938720,-1
2023-01-06,2940.421387,3022.823317,2929.846428,3021.404128,2488376,0,0,1,27.584002,68.245999,...,-100.0,101.721199,50.144278,inf,NaN,0,92.976890,8.744309,101.721199,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,3626.118896,3646.933000,3567.327220,3603.360828,2413058,1,0,1,75.363698,69.533260,...,-100.0,79.605780,79.958045,inf,NaN,0,79.605780,55.425432,24.180349,1
2023-12-26,3599.141357,3635.601635,3594.020653,3622.183949,1285231,1,0,1,66.955596,69.829550,...,-100.0,41.580982,80.543941,inf,NaN,0,41.580982,9.482739,32.098244,1


In [350]:
signals = ['RSI_Signal', 'BB_Signal', 'MACD_Signal', 'SO_Signal', 'ATR_Signal','ADX_Signal']
buys = []
sells =[]


for col in signals:
    counts = df[col].value_counts()
    n_buys = counts.get(1, 0)
    n_sells = counts.get(-1, 0)
    buys.append(n_buys)
    sells.append(n_sells)

res = pd.DataFrame({'Signal': signals, 'Buy': buys, 'Sell': sells})
res


,Signal,Buy,Sell
0,RSI_Signal,41,22
1,BB_Signal,18,6
2,MACD_Signal,143,116
3,SO_Signal,115,130
4,ATR_Signal,126,120
5,ADX_Signal,0,0
